#### **Import Libraries and Configuration**

In [54]:
!pip install --upgrade jupyter

Defaulting to user installation because normal site-packages is not writeable


In [1]:
#Import required functions and classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched, predict 
from sahi.prediction import visualize_object_predictions
from sahi.postprocess.utils import ObjectPredictionList, has_match, merge_object_prediction_pair
from IPython.display import Image
from numpy import asarray
import cv2
import os
import time
from PIL import Image
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult
from pathlib import Path
from multiprocessing import Pool, cpu_count
import torch
from torchvision.ops import nms, clip_boxes_to_image

In [2]:
# Download YOLOv8-S model to 'models/yolov8s.pt'
yolov8_model_path = 'models/yolov8/last.pt'

In [9]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.25,
    device="cuda:0", # or 'cpu'
)

In [5]:
def get_slice_parameters(object_density, slice_size):
    
    if object_density >= 50:
        #slice_size = min_dim // 4
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        return

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [6]:
def nms_merge(predictions, iou_threshold=0.5):
    if not predictions:
        return []
    boxes = torch.tensor([p.bbox.to_xyxy() for p in predictions])
    scores = torch.tensor([p.score.value for p in predictions])
    keep_indices = nms(boxes, scores, iou_threshold).tolist()
    #keep_indices = clip_boxes_to_image(boxes, 32).tolist()
    return [predictions[i] for i in keep_indices]

In [12]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

In [13]:
def merged_preds_to_coco(preds, image_id):
    coco_predictions = []
    for pred in preds:
        try:
            coco = pred.to_coco_prediction(image_id=image_id)
            coco_dict = {
                "image_id": coco.image_id,
                "bbox": coco.bbox,
                "score": coco.score,
                "category_id": coco.category_id,
                "segmentation": coco.segmentation,
                "iscrowd": coco.iscrowd,
                "area": coco.area,
            }
            # Optionally include category_name (not used in COCO eval)
            # coco_dict["category_name"] = coco.category_name

            # Only add if bbox is valid (non-empty)
            if coco_dict["bbox"]:
                coco_predictions.append(coco_dict)
        except Exception as e:
            print(f"Failed to convert to COCO dict: {e}")
    return coco_predictions


In [39]:
def truncated_nms_merge_latest(predictions: torch.Tensor,
                        IOUt: float = 0.55,   # Intersection-over-Union truncation threshold
                        IOIt: float = 0.35,   # Inside-intersection ratio threshold
                        IOOt: float = 0.20):  # Outside-intersection ratio threshold
    """
    Perform truncated non-maximum suppression (NMS) exactly per the provided algorithm.

    Args:
        predictions: Tensor of shape [num_boxes, 5] where each row is (x1, y1, x2, y2, score)
        IOUt: IoU threshold used outside/inside the truncated region (> / ≥ respectively)
        IOIt: Lower ratio threshold marking the start of the truncated region
        IOOt: Upper ratio threshold marking the end of the truncated region

    Returns:
        List[int]: indices of boxes kept after truncated NMS (selection order).
    """
    if predictions.numel() == 0:
        return []

    boxes = predictions[:, :4]
    scores = predictions[:, 4]

    # Ensure valid areas and avoid divide-by-zero
    eps = 1e-9
    w = (boxes[:, 2] - boxes[:, 0]).clamp(min=0)
    h = (boxes[:, 3] - boxes[:, 1]).clamp(min=0)
    areas = (w * h).clamp(min=eps)

    # Implement m ← argmax S by sorting once and popping the head each round
    order = scores.argsort(descending=True)
    keep = []

    while order.numel() > 0:
        m = order[0]
        M = boxes[m]
        area_M = areas[m]
        keep.append(m.item())

        if order.numel() == 1:
            break

        idxs = order[1:]
        R = boxes[idxs]

        # Intersection(M, b_i)
        xx1 = torch.maximum(R[:, 0], M[0])
        yy1 = torch.maximum(R[:, 1], M[1])
        xx2 = torch.minimum(R[:, 2], M[2])
        yy2 = torch.minimum(R[:, 3], M[3])
        iw = (xx2 - xx1).clamp(min=0)
        ih = (yy2 - yy1).clamp(min=0)
        inter = iw * ih

        # Ratios per algorithm
        ratio1 = inter / area_M            # intersection(M, b_i) / area(M)
        ratio2 = inter / areas[idxs]       # intersection(M, b_i) / area(b_i)

        # Truncated region: condition1 OR condition2
        condition1 = (ratio1 > IOIt) & (ratio2 <= IOOt)
        condition2 = (ratio1 <= IOOt) & (ratio2 > IOIt)
        truncated_region = condition1 | condition2

        # IoU(M, b_i)
        union = area_M + areas[idxs] - inter
        iou = inter / union.clamp(min=eps)

        # Suppress rules exactly as in the pseudocode:
        #  - Outside truncated region: suppress if IoU >  IOUt
        #  - Inside  truncated region: suppress if IoU >= IOUt
        drop = (~truncated_region & (iou >  IOUt)) | (truncated_region & (iou >= IOUt))

        # Keep only boxes not suppressed for the next rounds
        order = order[1:][~drop]

    return keep


In [14]:
import torch

def truncated_nms_merge_refractored(predictions: torch.Tensor,
                        IOUt: float = 0.5,   # Intersection‑over‑Union truncation threshold
                        IOIt: float = 0.4,   # Inside‑intersection ratio threshold
                        IOOt: float = 0.3):  # Outside‑intersection ratio threshold
    """
    Perform truncated non‑maximum suppression (NMS).  Boxes that overlap significantly
    with the current best box are suppressed using two different criteria:
      – ratio1 = intersection(M, b_i) / area(M)
      – ratio2 = intersection(M, b_i) / area(b_i)
    If either ratio lies in the “truncated” range defined by IOIt and IOOt, a slightly
    different IoU condition is used to decide whether to drop the candidate.

    Args:
        predictions: Tensor of shape [num_boxes, 5] where each row is (x1, y1, x2, y2, score)
        IOUt: IoU threshold used for boxes outside the truncated region (> IOUt causes suppression)
        IOIt: Lower ratio threshold marking the start of the truncated region
        IOOt: Upper ratio threshold marking the end of the truncated region

    Returns:
        List[int]: indices of boxes kept after truncated NMS.
    """
    boxes = predictions[:, :4]
    scores = predictions[:, 4]
    areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    indices = torch.arange(len(boxes))
    keep = []

    while indices.numel() > 0:
        # pick the box with the highest confidence
        m_idx = indices[scores[indices].argmax()]
        M = boxes[m_idx]
        area_M = areas[m_idx]

        keep.append(m_idx.item())

        # remove the selected box from further consideration
        indices = indices[indices != m_idx]
        if indices.numel() == 0:
            break

        # compute intersection with all remaining boxes
        remaining_boxes = boxes[indices]
        xx1 = torch.maximum(remaining_boxes[:, 0], M[0])
        yy1 = torch.maximum(remaining_boxes[:, 1], M[1])
        xx2 = torch.minimum(remaining_boxes[:, 2], M[2])
        yy2 = torch.minimum(remaining_boxes[:, 3], M[3])
        w = torch.clamp(xx2 - xx1, min=0)
        h = torch.clamp(yy2 - yy1, min=0)
        inter = w * h

        # ratio1 = intersection area / area of M
        ratio1 = inter / area_M
        # ratio2 = intersection area / area of candidate
        ratio2 = inter / areas[indices]

        # identify boxes in truncated region: either ratio1 > IOIt and ratio2 <= IOOt
        # or ratio1 <= IOOt and ratio2 > IOIt
        condition1 = (ratio1 > IOIt) & (ratio2 <= IOOt)
        condition2 = (ratio1 <= IOOt) & (ratio2 > IOIt)
        truncated_region = condition1 | condition2

        # compute IoU for each remaining box
        union = area_M + areas[indices] - inter
        iou = inter / union

        # decide which boxes to suppress
        # outside truncated region: drop if IoU > IOUt
        # inside truncated region: drop if IoU >= IOUt
        suppress = (~truncated_region & (iou > IOUt)) | (truncated_region & (iou >= IOUt))

        # keep only those that are not suppressed
        indices = indices[~suppress]

    return keep


In [15]:
def truncated_nms_merge(
    predictions: torch.Tensor,
    match_metric: str = "IOU",
    IOUt: float = 0.5,  # Truncation IoU threshold for keeping boxes
    IOIt: float = 0.4,  # Inside IoU threshold
    IOOt: float = 0.3   # Outside IoU threshold
):
    """
    Apply truncated non-maximum suppression to avoid detecting too many
    overlapping bounding boxes for a given object, with added truncation logic.

    Args:
        predictions (tensor): The location preds for the image along with the class scores, Shape: [num_boxes, 5].
        match_metric (str): IOU or IOS (Intersection over Area or Intersection over Union)
        match_threshold (float): The overlap threshold for match metric.
        IOUt (float): Intersection over Union threshold for truncation (threshold to keep boxes)
        IOIt (float): Inside Intersection over Union threshold (threshold to keep inside box)
        IOOt (float): Outside Intersection over Union threshold (threshold for outside box)

    Returns:
        List: A list of filtered indexes
    """
    # Extract coordinates for every prediction box present in P
    x1 = predictions[:, 0]
    y1 = predictions[:, 1]
    x2 = predictions[:, 2]
    y2 = predictions[:, 3]

    # Extract the confidence scores
    scores = predictions[:, 4]

    # Calculate area of every box
    areas = (x2 - x1) * (y2 - y1)

    # Sort the prediction boxes in P according to their confidence scores
    order = scores.argsort()

    # Initialize an empty list for filtered prediction boxes
    keep = []

    while len(order) > 0:
        # Extract the index of the prediction with the highest score (S)
        idx = order[-1]

        # Push S in filtered predictions list
        keep.append(idx.tolist())

        # Remove S from P
        order = order[:-1]

        # Sanity check
        if len(order) == 0:
            break

        # Select coordinates of remaining boxes according to the indices in order
        xx1 = torch.index_select(x1, dim=0, index=order)
        xx2 = torch.index_select(x2, dim=0, index=order)
        yy1 = torch.index_select(y1, dim=0, index=order)
        yy2 = torch.index_select(y2, dim=0, index=order)

        # Find the coordinates of the intersection boxes
        xx1 = torch.max(xx1, x1[idx])
        yy1 = torch.max(yy1, y1[idx])
        xx2 = torch.min(xx2, x2[idx])
        yy2 = torch.min(yy2, y2[idx])

        # Find height and width of the intersection boxes
        w = xx2 - xx1
        h = yy2 - yy1

        # Take max with 0.0 to avoid negative width and height
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)

        # Find the intersection area
        inter = w * h

        # Find the areas of the remaining boxes according to the indices in order
        rem_areas = torch.index_select(areas, dim=0, index=order)

        # Calculate the match metric value (IoU or IoS)
        if match_metric == "IOU":
            # Find the union of every prediction T in P with the prediction S
            union = (rem_areas - inter) + areas[idx]
            match_metric_value = inter / union
        elif match_metric == "IOS":
            # Find the smaller area of every prediction T in P with the prediction S
            smaller = torch.min(rem_areas, areas[idx])
            match_metric_value = inter / smaller
        else:
            raise ValueError("Invalid match_metric. Choose either 'IOU' or 'IOS'.")

        # Add the condition for Truncated NMS:
        # - Keep boxes with IoU below the truncation threshold (IOUt)
        # - Handle inside (IOIt) and outside (IOOt) intersection thresholds
        mask = (match_metric_value < IOUt)  # Keep boxes with IoU below threshold
        #print(mask)

        # Apply the truncated NMS conditions
        for i, m in enumerate(mask):
            # Condition for truncated NMS (condition 1 and 2 from original pseudocode)
            iou = match_metric_value[i]
            condition1 = iou > IOIt and iou <= IOOt
            condition2 = iou <= IOOt and iou > IOIt

            condition = condition1 or condition2
            if not condition1:
              if iou <= IOUt and not condition:
                  mask[i] = True  # Keep box if condition is met
                  #keep.append(idx.tolist())
              else:
                  mask[i] = False  # Remove box if condition is met
            else:
                if iou >= IOUt:
                    mask[i] = False  # Remove box if condition is met
                else:
                    mask[i] = True  # Keep box if condition is met

        # Filter out the boxes based on the updated mask
        order = order[mask]

    print("Total Valid prediction: ", len(keep))
        
    return keep

In [23]:
def predict_fine_sliced_images_refactored_nms(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\nRunning fine slicing prediction on {len(image_files)} image(s)...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)

        img_id = get_image_id(data, filename_wo_ext) if dataset_json_path else None
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                coarse_preds = base_pred.object_prediction_list
                object_density = len(coarse_preds)
                print("Object Density:", object_density)

                slice_params = get_slice_parameters(object_density, base_slice_size)

                iteration_time = time_end

                preds = coarse_preds.copy()

                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="NMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.1,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    fine_preds = sliced_pred.object_prediction_list
                    preds.extend(fine_preds)
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    #preds = base_pred.object_prediction_list

                # Offset bounding boxes back to original image space
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)

                total_prediction_count += len(preds)
                total_prediction_time += iteration_time

        print("________________________________________________")

        # Merge all predictions for image
        merged_preds = nms_merge(all_object_predictions, iou_threshold=0.5)
        #object_prediction_list = ObjectPredictionList(all_object_predictions)
        #merged_keep = truncated_nms_merge(object_prediction_list.totensor())
        #selected_object_predictions = object_prediction_list[merged_keep].tolist()
        #if not isinstance(selected_object_predictions, list):
        #    selected_object_predictions = [selected_object_predictions]
        
        # Convert merged predictions to COCO format
        if dataset_json_path:
            coco_preds = merged_preds_to_coco(merged_preds, img_id)
            all_coco_preds.extend(coco_preds)
            
        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=merged_preds,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\nCompleted {len(image_files)} images.")
    print("Total Prediction Count (Before NMS Merge): ", grand_total_predictions)
    print("Total Prediction Count (After Final NMS Merge): ", len(all_coco_preds))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [20]:
def predict_fine_sliced_images_refactored(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\nRunning fine slicing prediction on {len(image_files)} image(s)...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)

        img_id = get_image_id(data, filename_wo_ext) if dataset_json_path else None
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                coarse_preds = base_pred.object_prediction_list
                object_density = len(coarse_preds)
                print("Object Density:", object_density)

                slice_params = get_slice_parameters(object_density, base_slice_size)

                iteration_time = time_end

                preds = coarse_preds.copy()

                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="NMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    fine_preds = sliced_pred.object_prediction_list
                    preds.extend(fine_preds)
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    #preds = base_pred.object_prediction_list

                # Offset bounding boxes back to original image space
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)

                total_prediction_count += len(preds)
                total_prediction_time += iteration_time

        print("________________________________________________")

        # Merge all predictions for image
        #merged_preds = nms_merge(all_object_predictions, iou_threshold=0.5)
        object_prediction_list = ObjectPredictionList(all_object_predictions)
        merged_keep = truncated_nms_merge_latest(object_prediction_list.totensor())
        selected_object_predictions = object_prediction_list[merged_keep].tolist()
        if not isinstance(selected_object_predictions, list):
            selected_object_predictions = [selected_object_predictions]
        
        # Convert merged predictions to COCO format
        if dataset_json_path:
            coco_preds = merged_preds_to_coco(selected_object_predictions, img_id)
            all_coco_preds.extend(coco_preds)
            
        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=selected_object_predictions,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\nCompleted {len(image_files)} images.")
    print("Total Prediction Count (Before NMS Merge): ", grand_total_predictions)
    print("Total Prediction Count (After Final NMS Merge): ", len(all_coco_preds))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [40]:
def predict_multilevel_fine_sliced_images(input_folder, dataset_json_path, detection_model, base_slice_size=512):

    print("\n***********************************************")
    print("Adaptive MultiLevel SAHI Prediction")
    print("*************************************************")
    
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\nRunning fine slicing prediction on {len(image_files)} image(s)...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name: ", filename_wo_ext)
        print("*****************************************")
        
        img_id = get_image_id(data, filename_wo_ext) if dataset_json_path else None
        all_object_predictions = []

        #Base prediction
        t0 = time.time()
        intial_prediction = get_prediction(image_path, detection_model)
        t_base = (time.time() - t0)
        print(f"Base pred time: {t_base * 1000:.2f} ms")
        total_prediction_time += t_base
        print(f"Total Based pred time: {total_prediction_time * 1000:.2f} ms")
        full_preds = intial_prediction.object_prediction_list
        full_object_density = len(full_preds)
        print("Full Image Object Density:", full_object_density)

        for pred in full_preds:
            all_object_predictions.append(pred)
            
        total_prediction_count += len(all_object_predictions)
    
        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image Dimension:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                coarse_preds = base_pred.object_prediction_list
                object_density = len(coarse_preds)
                print("Slice Object Density:", object_density)
                
                slice_params = get_slice_parameters(object_density, base_slice_size)
            
                iteration_time = time_end
                
                preds = coarse_preds.copy()
                
                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="NMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    fine_preds = sliced_pred.object_prediction_list
                    print("Pre Adaptive Slicing: ", len(preds))
                    preds.extend(fine_preds)
                    print("Post Adaptive Slicing: ", len(preds))
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))

                total_prediction_time += iteration_time
                
                #Offset bounding boxes back to original image space
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)
        
                total_prediction_count += len(preds)

        print("________________________________________________")

        print("Total Prediction Count (Crop+Slice): ", len(all_object_predictions))
        
        # Merge all predictions for image
        #merged_preds = apply_nms(all_object_predictions, iou_threshold=0.5)
        
        object_prediction_list = ObjectPredictionList(all_object_predictions)
        merged_keep = truncated_nms_merge_latest(object_prediction_list.totensor())
        selected_object_predictions = object_prediction_list[merged_keep].tolist()
        if not isinstance(selected_object_predictions, list):
           selected_object_predictions = [selected_object_predictions]
        
        # Convert merged predictions to COCO format
        if dataset_json_path:
            coco_preds = merged_preds_to_coco(selected_object_predictions, img_id)
            all_coco_preds.extend(coco_preds)
            
        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=selected_object_predictions,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\nCompleted {len(image_files)} images.")
    print("Total Prediction Count (Before NMS Merge): ", grand_total_predictions)
    print("Total Prediction Count (After Final NMS Merge): ", len(selected_object_predictions))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


#### **Subset of 50 images (V1)**

In [25]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 70.22 ms
Object Density: 23
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 121
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Sliced Prediction time is: 87.66 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.84 ms
Object Density: 8
Prediction time is: 7.84 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.89 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes 

In [26]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp834/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.92s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.260
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.482
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [20]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.91 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 84.26 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.75 ms
Object Density: 3
Prediction time is: 7.75 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.86 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding

In [21]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp833/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.43s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 71.42 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 83.17 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.74 ms
Object Density: 3
Prediction time is: 7.74 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.74 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding

In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp832/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.43s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [10]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394


/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Initial Prediction time is: 1239.41 ms
Object Density: 23
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 121
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Sliced Prediction time is: 88.85 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.98 ms
Object Density: 8
Prediction time is: 7.98 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 8.01 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 85
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count

In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp831/result.json'

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.92s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.260
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.482
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.205
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [29]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.38 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 82.10 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.51 ms
Object Density: 3
Prediction time is: 7.51 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.58 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding

In [30]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp425/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.43s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [25]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.60 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 82.50 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.50 ms
Object Density: 3
Prediction time is: 7.50 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.65 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding

In [26]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp424/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.43s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [19]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.25 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 82.20 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.54 ms
Object Density: 3
Prediction time is: 7.54 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.85 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding

In [20]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp422/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.47s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.248
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.313
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.100
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.190
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 8.68 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 80.83 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.47 ms
Object Density: 3
Prediction time is: 7.47 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.59 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding 

In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp421/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.43s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [27]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.40 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  66
Final Bounding Box Count (OptNMS): 48
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 204.48 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.64 ms
Object Density: 3
Prediction time is: 7

In [28]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp405/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.55s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.219
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.317
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.197
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [24]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 1631.35 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 28
POST PROCESS:  NMS
Original Prediction Count 112
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  45
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 205.97 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.69 ms
Object Density: 3
Prediction time is: 7.69 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 8.07 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Bo

In [25]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp404/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.54s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.222
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [29]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.42 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  56
Final Bounding Box Count (OptNMS): 55
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Sliced Prediction time is: 552.89 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.62 ms
Object Density: 3
Prediction time is: 7.62 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.80 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  

In [30]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp406/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.56s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.248
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.149
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.332
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [32]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.05 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 70
Final Bounding Box Count (NMS):  51
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Sliced Prediction time is: 540.78 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.50 ms
Object Density: 3
Prediction time is: 7.50 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.55 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 91
POST PROCESS:  NMS
Original Prediction Count 40
Final Bounding B

In [33]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16 (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp407/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.53s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.248
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.149
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.332
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [44]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.42 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 81.34 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.45 ms
Object Density: 3
Prediction time is: 7.45 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.54 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:

In [46]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp402/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.50s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.100
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS) --> OptNMS -> NMS- Union
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 8.71 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 79.83 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.36 ms
Object Density: 3
Prediction time is: 7.36 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.61 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding 

In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp392/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.43s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.099
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [66]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 11.25 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 83.92 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.71 ms
Object Density: 3
Prediction time is: 7.71 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.77 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:

In [67]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp417/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.45s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [15]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.33 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 80.78 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.42 ms
Object Density: 3
Prediction time is: 7.42 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.60 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:

In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp393/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.45s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.246
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.098
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.58 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  23
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Sliced Prediction time is: 81.38 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.51 ms
Object Density: 3
Prediction time is: 7.51 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.54 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:

In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp394/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.75s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.183
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.220
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.427
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [48]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v6/images'
json_path = "./subset_visdrone_test_data_50_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 50 image(s)...
*****************************************
File Name 9999938_00000_d_0000283
Cropped Image: 0 0 700 394
Initial Prediction time is: 10.74 ms
Object Density: 18
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 68
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  4
Sliced Prediction time is: 81.50 ms
Cropped Image: 700 0 1400 394
Initial Prediction time is: 7.58 ms
Object Density: 3
Prediction time is: 7.58 ms
Cropped Image: 0 394 700 788
Initial Prediction time is: 7.63 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 58
Final Bounding

In [49]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './sliced_predictions/exp403/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.47s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.248
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.313
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.100
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.190
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [50]:
result_predict_nms = predict(source='./test_subsets/v6',
                         dataset_json_path = './subset_visdrone_test_data_50_v1.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/50 [00:00<?, ?it/s]

Image Name: 9999938_00000_d_0000283
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


Performing inference on images:   2%|▏         | 1/50 [00:00<00:34,  1.40it/s]

Original Prediction Count 285
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  40
Final Bounding Box Count (NMS):  38
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  10
Prediction time is: 581.80 ms
Image Name: 9999938_00000_d_0000109
Image Size:  (1400, 788)
Sliced Boxes Count: 60
POST PROCESS:  NMS
Performing prediction on 60 slices.


...

Performing inference on images:  98%|█████████▊| 49/50 [00:33<00:00,  1.80it/s]

Original Prediction Count 275
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  17
Prediction time is: 255.17 ms
Image Name: 9999973_00000_d_0000094
Image Size:  (1916, 1078)
Sliced Boxes Count: 112
POST PROCESS:  NMS
Performing prediction on 112 slices.


Performing inference on images: 100%|██████████| 50/50 [00:34<00:00,  1.44it/s]

Original Prediction Count 69
Final Bounding Box Count (NMS):  13
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  6
Prediction time is: 918.24 ms


Prediction results are successfully exported to runs/predict/exp275
Model loaded in 0.3050975799560547 seconds.
Slicing performed in 0.07773113250732422 seconds.
Prediction performed in 28.40929675102234 seconds.
Exporting performed in 0.09507060050964355 seconds.


In [51]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_50_v1.json' --result_json_path './runs/predict/exp275/result.json'

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.54s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.547
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 15 images (V5)**

In [41]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-NMS) --> latest truncated NMS 
#IOUt: float = 0.55,   # Intersection-over-Union truncation threshold
#IOIt: float = 0.35,   # Inside-intersection ratio threshold
#IOOt: float = 0.20

source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 29.89 ms
Total Based pred time: 29.89 ms
Full Image Object Density: 45
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.48 ms
Slice Object Density: 3
Prediction time is: 9.48 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 8.96 ms
Slice Object Density: 22
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 34
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [42]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp1053/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.254
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.162
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.304
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.393
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [37]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-NMS) --> latest truncated NMS 
#IOUt: float = 0.60,   # Intersection-over-Union truncation threshold
#IOIt: float = 0.40,   # Inside-intersection ratio threshold
#IOOt: float = 0.20

source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 31.28 ms
Total Based pred time: 31.28 ms
Full Image Object Density: 45
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.58 ms
Slice Object Density: 3
Prediction time is: 9.58 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 9.09 ms
Slice Object Density: 22
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 34
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [38]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp1052/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.302
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.393
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.190
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [28]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-NMS) --> latest truncated NMS
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 15 image(s)...
*****************************************
File Name:  9999963_00000_d_0000057
*****************************************
Base pred time: 29.30 ms
Total Based pred time: 29.30 ms
Full Image Object Density: 45
Cropped Image Dimension: 0 0 700 525
Initial Prediction time is: 9.10 ms
Slice Object Density: 3
Prediction time is: 9.10 ms
Cropped Image Dimension: 700 0 1400 525
Initial Prediction time is: 8.86 ms
Slice Object Density: 22
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 34
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding 

In [29]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp1049/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.25s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.250
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.369
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [21]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-NMS) --> latest truncated NMS
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.19 ms
Object Density: 3
Prediction time is: 11.19 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.13 ms
Object Density: 22
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 34
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 110.42 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.39 ms
Object Density: 9
Prediction time is: 8.39 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.40 ms
Object Density: 2
Prediction time is: 8.40 ms
____________

In [22]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp1046/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.182
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.247
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [17]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-NMS) --> NMS
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 10.55 ms
Object Density: 3
Prediction time is: 10.55 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.01 ms
Object Density: 22
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 34
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 110.66 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.47 ms
Object Density: 9
Prediction time is: 8.47 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.44 ms
Object Density: 2
Prediction time is: 8.44 ms
____________

In [18]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp1045/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.105
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.182
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.247
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [64]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-NMS) --> NMS
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 70.30 ms
Object Density: 3
Prediction time is: 70.30 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.81 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  26
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 109.52 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.73 m

In [65]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp416/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.104
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.152
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.240
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [31]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 11.57 ms
Object Density: 3
Prediction time is: 11.57 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.94 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 108.14 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.26 ms
Object Density: 7
Prediction time is: 8.26 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.20 ms
Object Density: 2
Prediction time is: 8.20 ms
___________________________________

In [32]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp426/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.209
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.272
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [36]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 9.85 ms
Object Density: 3
Prediction time is: 9.85 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.97 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  NMS
Original Prediction Count 31
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  18
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 108.21 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.32 ms
Object Density: 7
Prediction time is: 8.32 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.32 ms
Object Density: 2
Prediction time is: 8.32 ms
_____________________________________

In [38]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp409/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.209
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.272
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [40]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.16 ms
Object Density: 3
Prediction time is: 12.16 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.95 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 12
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  26
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 109.35 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.39 m

In [41]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp410/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.206
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.269
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.173
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [44]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 9.79 ms
Object Density: 3
Prediction time is: 9.79 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 8.93 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 35
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 9
Sliced Prediction time is: 233.83 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.36 ms
Object Density: 7
Prediction time is: 8.36 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.32 ms
Object Density: 2
Prediction time is: 8.32 ms
________________________________________________


In [45]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp412/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.206
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [47]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.13 ms
Object Density: 3
Prediction time is: 12.13 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.18 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 35
POST PROCESS:  NMS
Original Prediction Count 38
Final Bounding Box Count (NMS):  25
Final Bounding Box Count (NMS):  6
Sliced Prediction time is: 235.91 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.42 ms
Object Density: 7
Prediction time is: 8.42 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.37 ms
Object Density: 2
Prediction time is: 8.37 ms
________________________________________________
Total Valid prediction:  43
*****************************************
File Name 9999963_000

In [48]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp413/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.126
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.312
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [70]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 128

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.23 ms
Object Density: 3
Prediction time is: 12.23 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.54 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 35
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 9
Sliced Prediction time is: 239.82 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.57 ms
Object Density: 7
Prediction time is: 8.57 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.54 ms
Object Density: 2
Prediction time is: 8.54 ms
_______________________________________________

In [71]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp419/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.126
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [49]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.25 ms
Object Density: 3
Prediction time is: 12.25 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.19 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 130
POST PROCESS:  NMS
Original Prediction Count 25
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 767.68 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.46 ms
Object Density: 7
Prediction time is: 8.46 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.38 ms
Object Density: 2
Prediction time is: 8.38 ms
________________________________________________
Total Valid prediction:  36
*****************************************
File Name 9999963_0000

In [50]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp414/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.222
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [52]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.13 ms
Object Density: 3
Prediction time is: 12.13 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.29 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 130
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 23
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 767.79 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.44 ms
Object Density: 7
Prediction time is: 8.44 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.41 ms
Object Density: 2
Prediction time is: 8.41 ms
________________________________________________

In [53]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp415/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.18s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.222
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [68]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './test_subsets/v5/images'
json_path = "./subset_visdrone_test_data_15_v5.json"
slice_size = 64

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 15 image(s)...
*****************************************
File Name 9999963_00000_d_0000057
Cropped Image: 0 0 700 525
Initial Prediction time is: 12.14 ms
Object Density: 3
Prediction time is: 12.14 ms
Cropped Image: 700 0 1400 525
Initial Prediction time is: 9.34 ms
Object Density: 20
********* Slice Parameters ***********
Slice Width:  64
Slice Height:  64
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 130
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 23
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 772.22 ms
Cropped Image: 0 525 700 1050
Initial Prediction time is: 8.59 ms
Object Density: 7
Prediction time is: 8.59 ms
Cropped Image: 700 525 1400 1050
Initial Prediction time is: 8.49 ms
Object Density: 2
Prediction time is: 8.49 ms
________________________________________________

In [69]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_15_v5.json' --result_json_path './sliced_predictions/exp418/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.223
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.142
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.319
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Single Image**

In [13]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    #sliced_detection_model = detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 1 image(s)...
*****************************************
File Name 0000006_06773_d_0000018
Cropped Image: 0 0 680 382
Initial Prediction time is: 8.59 ms
Object Density: 27
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 101
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  14
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  5
Sliced Prediction time is: 81.56 ms
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.39 ms
Object Density: 8
Prediction time is: 7.39 ms
Cropped Image: 0 382 680 764
Initial Prediction time is: 7.36 ms
Object Density: 16
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap H

In [14]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp795/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.416
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.742
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.493
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.933
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.747
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.413
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.484
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [15]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './single_test/test_image/images'
json_path = "./subset_visdrone_test_data_46.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 1 image(s)...
*****************************************
File Name 0000011_01955_d_0000004
Cropped Image: 0 0 680 382
Initial Prediction time is: 9.98 ms
Object Density: 32
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Original Prediction Count 123
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  10
Sliced Prediction time is: 87.22 ms
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.92 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 10

In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_46.json' --result_json_path './sliced_predictions/exp796/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.161
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.295
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './single_test/test_image/images'
json_path = "./subset_visdrone_test_data_46.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_nms(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


Running fine slicing prediction on 1 image(s)...
*****************************************
File Name 0000011_01955_d_0000004
Cropped Image: 0 0 680 382
Initial Prediction time is: 11.24 ms
Object Density: 32
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  TruncatedNMS
Original Prediction Count 123
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  39
Final Bounding Box Count (Truncated NMS):  24
Sliced Prediction time is: 350.56 ms
Cropped Image: 680 0 1360 382
Initial Prediction time is: 8.20 ms
Object Density: 62
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 10
POST PROCESS:  TruncatedNMS
Original Prediction Count 201
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  57
Final Bounding Box Count (Truncated NM

In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_46.json' --result_json_path './sliced_predictions/exp797/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.161
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.040
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.292
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [45]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    #sliced_detection_model = detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 1 image(s)...
*****************************************
File Name:  0000006_06773_d_0000018
*****************************************
Base pred time: 25.71 ms
Total Based pred time: 25.71 ms
Full Image Object Density: 41
Cropped Image Dimension: 0 0 680 382
Initial Prediction time is: 8.38 ms
Slice Object Density: 17
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 43
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  16
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Sliced Prediction time is: 62.21 ms
Pre Adaptive Slicing:  17
Post Adaptive

In [48]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp1055/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.447
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.792
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.532
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.976
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.804
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.514
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **High 50 images**

In [30]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
source_folder = './custom_utils/testdev_subsets/high/v1/images'
json_path = "./custom_utils/annotations/subset_visdrone_test_data_50_high_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 50 image(s)...
*****************************************
File Name:  9999938_00000_d_0000372
*****************************************
Base pred time: 25.96 ms
Total Based pred time: 25.96 ms
Full Image Object Density: 38
Cropped Image Dimension: 0 0 700 394
Initial Prediction time is: 8.42 ms
Slice Object Density: 32
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 111
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Sliced Prediction time is: 86.13 ms
Pre Adaptive Slicing:  32
Post Adaptive Slicing:  98
Cropped Image Dimension: 700 0 1400 394
Initial Predi

In [31]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './custom_utils/annotations/subset_visdrone_test_data_50_high_v1.json' --result_json_path './sliced_predictions/exp1050/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.94s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.234
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.400
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.625
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.213
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [34]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
#IOUt: float = 0.60,   # Intersection-over-Union truncation threshold
#IOIt: float = 0.40,   # Inside-intersection ratio threshold
#IOOt: float = 0.20

source_folder = './custom_utils/testdev_subsets/high/v1/images'
json_path = "./custom_utils/annotations/subset_visdrone_test_data_50_high_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 50 image(s)...
*****************************************
File Name:  9999938_00000_d_0000372
*****************************************
Base pred time: 25.47 ms
Total Based pred time: 25.47 ms
Full Image Object Density: 38
Cropped Image Dimension: 0 0 700 394
Initial Prediction time is: 8.55 ms
Slice Object Density: 32
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 111
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Sliced Prediction time is: 85.35 ms
Pre Adaptive Slicing:  32
Post Adaptive Slicing:  98
Cropped Image Dimension: 700 0 1400 394
Initial Predi

In [35]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './custom_utils/annotations/subset_visdrone_test_data_50_high_v1.json' --result_json_path './sliced_predictions/exp1051/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.01s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.235
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.412
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.636
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.226
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [43]:
# Example usage: Adaptive-Optimized-NMS Take 2(Adaptive-OptNMS)
# Using Different IoU threshold
#IOUt: float = 0.55,   # Intersection-over-Union truncation threshold
#IOIt: float = 0.35,   # Inside-intersection ratio threshold
#IOOt: float = 0.20

source_folder = './custom_utils/testdev_subsets/high/v1/images'
json_path = "./custom_utils/annotations/subset_visdrone_test_data_50_high_v1.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_multilevel_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              
)


***********************************************
Adaptive MultiLevel SAHI Prediction
*************************************************

Running fine slicing prediction on 50 image(s)...
*****************************************
File Name:  9999938_00000_d_0000372
*****************************************
Base pred time: 26.16 ms
Total Based pred time: 26.16 ms
Full Image Object Density: 38
Cropped Image Dimension: 0 0 700 394
Initial Prediction time is: 8.44 ms
Slice Object Density: 32
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Confidence Score:  0.3
Original Prediction Count 111
Final Bounding Box Count (NMS):  33
Final Bounding Box Count (NMS):  20
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  8
Sliced Prediction time is: 85.91 ms
Pre Adaptive Slicing:  32
Post Adaptive Slicing:  98
Cropped Image Dimension: 700 0 1400 394
Initial Predi

In [44]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './custom_utils/annotations/subset_visdrone_test_data_50_high_v1.json' --result_json_path './sliced_predictions/exp1054/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.98s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.235
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.413
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.625
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.222
 Average Precision  (AP) @[ IoU=0.50:0.95 | a